In [ ]:
import cv2
from ultralytics import YOLO
import torch
import numpy as np
from twilio.rest import Client
from findpolice import find_optimum_police
from sort.sort import Sort
import datetime
import threading

In [ ]:
model = YOLO('.../best.pt')  
model2 = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained = True) 
tracker = Sort()

In [ ]:
thread_lock = threading.Lock()
update_thread = None

def find_police_(location):
    police,  police_contact = find_optimum_police(location, show_info = False, show_directions = False)

    if police_contact is not None:
        # SMS Police 
        message = client.messages.create(from_ = twilio_phone_number,
                                        body = f'ALERT! Accident Detected at coordinates - {location}. Map Link: https://www.google.com/maps/place/{location[0]},{location[1]}',
                                        to = ' ')
        cv2.putText(frame, message.sid, (250, 250), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 0), 2)
    
    else:
        cv2.putText(frame, "No Police Contact found", (250, 250), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 0), 2)
        
    # # Calling Police
    # call_recipient_phone_number = police_contact
    # call = client.calls.create(
    # url = 'https://ginger-bear-4981.twil.io/assets/text_to_speech.mp3',
    # to = call_recipient_phone_number,
    # from_ = twilio_phone_number)
    # print(call.sid)

def find_police(location):
    global update_thread
    if update_thread is None or not update_thread.is_alive():
        update_thread = threading.Thread(target = find_police_, args = (location,), daemon = True)
        update_thread.start()
    else:
        with thread_lock:
            update_thread._args = (location,)


In [ ]:
# Main 
account_sid = ' '
auth_token = ' '
twilio_phone_number = ' '
client = Client(account_sid, auth_token)

cap = cv2.VideoCapture("../TestVideo.mp4")
count = 0
accident_ids = set()
last_reset_time = datetime.datetime.now()

while True:
    # current_time = datetime.datetime.now()
    # if current_time.day != last_reset_time.day:
    #     accident_ids = set() 
    #     last_reset_time = current_time

    ret, frame = cap.read()
    if ret is False: break
    count += 1
    if count%4 != 0:
        continue

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # frame = cv2.resize(frame, (1020, 500))

    results = np.array(model.predict(frame)[0].boxes.data) 

    accident_list = []
    for result in results:
        x1, y1, x2, y2 = map(int, result[:4])
        # cv2.rectangle(frame,(x1, y1), (x2, y2), (0,255,0), 2)
        accident_list.append([x1, y1, x2, y2])
    
    if accident_list:
        track_list = tracker.update(np.array(accident_list))
        for accident in track_list:
            x1, y1, x2, y2, id = map(int, accident)
            accident_frame = frame[y1:y2, x1:x2]
            if id not in accident_ids:
                if accident_frame.size:
                    results2 = model2(accident_frame).pandas().xyxy[0]
                    car_count = len(results2[results2['name'].isin(['car', 'truck', 'motorcycle'])])
                    if car_count > 1:
                        accident_ids.add(id)
                        gps_location = () # (Lat, Lng) - Fetched From a GPS Module (CCTV camera should be equipped with a GPS module)
                        find_police(gps_location)
                        cv2.putText(frame, str(id), ((x1+x2)//2, (y1+y2)//2), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 1)
                        cv2.putText(frame, "Accident Detected! ", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 0), 2)
                        
            else:
                cv2.putText(frame, "Accident Detected! ", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 0), 2)

    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)